In [1]:
# import os

# # 设置环境变量
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# # 打印环境变量以确认设置成功
# print(os.environ.get('HF_ENDPOINT'))

# import subprocess
# import os

# result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
# output = result.stdout
# for line in output.splitlines():
#     if '=' in line:
#         var, value = line.split('=', 1)
#         os.environ[var] = value

In [2]:
import json
import random
import re
import os
from datasets import load_dataset
from openai import OpenAI
from sklearn.metrics import accuracy_score, classification_report

In [3]:
# ==========================================
# 1. 配置 API  https://bailian.console.aliyun.com/
# ==========================================
API_KEY = "sk-"  # 注意：在生产环境中请勿硬编码，建议使用 os.getenv
MODEL_ID = "qwen3-max"

client = OpenAI(
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=API_KEY,
)

In [4]:
# ==========================================
# 2. 数据准备 (加载、采样、打乱)
# ==========================================
print("Loading dataset...")
try:
    # 尝试加载数据集
    protein_data = load_dataset('dnagpt/gene_lan_transfer', 'dna_protein_pair')
    # local_dataset_path = "./gene_lan_transfer" 
    # # 修改这里：第一个参数改为本地路径
    # protein_data = load_dataset(
    #     local_dataset_path,          # 👈 这里改成你的本地文件夹路径
    #     'dna_protein_pair',    # 配置名保持不变dna_sim_pair_50bp,dna_sim_pair_150bp
    #     trust_remote_code=True       # 👈 加上这个，允许执行本地文件夹里的加载脚本
    # )

    # 方法：直接从 CSV 文件加载（推荐，最简单）
    # 假设你的 CSV 文件有三列：sentence1, sentence2, label
    # csv_path = "protein_pair_sample_200.csv"
    
    # # 使用 load_dataset 直接读取本地 CSV
    # protein_data = load_dataset("csv", data_files=csv_path)

    
    ds = protein_data['train']
    
    # 分离数据
    data_label_0 = [item for item in ds if item['label'] == 0]
    data_label_1 = [item for item in ds if item['label'] == 1]
    
    # 随机采样 (各50个)
    random.seed(42)
    sample_num = 50  # 注意：总数是100，所以各取50
    sampled_0 = random.sample(data_label_0, sample_num)
    sampled_1 = random.sample(data_label_1, sample_num)
    
    # 合并并打乱
    combined_data = sampled_0 + sampled_1
    random.shuffle(combined_data)
    
    print(f"Data prepared: {len(combined_data)} pairs.")

except Exception as e:
    print(f"Error loading dataset: {e}")
    exit()

# 构建用于 Prompt 的 JSON List，并建立 ID -> Label 的映射用于后续验证
prompt_data_list = []
id_to_ground_truth = {}

for idx, item in enumerate(combined_data, 1):
    prompt_data_list.append({
        "id": idx,
        "seq_a": item['sentence1'],
        "seq_b": item['sentence2']
    })
    id_to_ground_truth[idx] = item['label']

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset './gene_lan_transfer' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


Loading dataset...
Data prepared: 100 pairs.


In [5]:
# # ==========================================
# # 3. 存储数据到文件 (包含 Label)
# # ==========================================
# import json

# # 定义文件名
# output_filename = "prompt_data_list.json"

# print(f"Saving {len(prompt_data_list)} items to {output_filename}...")

# # --- 关键修改：在保存前，把 label 从 id_to_ground_truth 中合并进去 ---
# # 这一步不需要动前面的代码，直接利用已有的 id_to_ground_truth 字典
# for item in prompt_data_list:
#     item_id = item['id']
#     if item_id in id_to_ground_truth:
#         item['label'] = id_to_ground_truth[item_id]

# # 写入文件
# with open(output_filename, "w", encoding="utf-8") as f:
#     # indent=2 让文件格式化（有缩进），方便人类阅读
#     # ensure_ascii=False 防止中文（如果有的话）被转义成乱码
#     json.dump(prompt_data_list, f, indent=2, ensure_ascii=False)

# print("Save complete (with labels).")

In [6]:
# ==========================================
# 3. 构建 Prompt (去规则化 / 纯 NLP 直觉版本)
# ==========================================

# System Prompt: 移除具体生物学规则，仅保留“加密/解密”的抽象逻辑
system_prompt = """You are an expert in "Abstract Sequence Alignment" and "Cipher Decryption".

1. The Concept:
You are evaluating whether a Target Sequence is a valid "decrypted" version of a Source Sequence.
* A **Valid Decryption** implies the information is perfectly preserved and transformed according to a hidden consistent pattern.
* A **Corrupted Decryption** contains errors, noise, or is completely unrelated to the source.

2. The Analogy:
* **DNA Sequence (Sentence 1)** is the **Source Ciphertext** (Raw Data).
* **Protein Sequence (Sentence 2)** is the **Target Plaintext** (Decoded Information).
* **Coding Relationship (Label 1)** is like a **Lossless Translation**: The hidden patterns in the Source map faithfully to the Target. The information flow is intact.
* **Non-Coding Relationship (Label 0)** is like a **Hallucination or Glitch**:
    - The Target sequence breaks the mapping logic found in the Source.
    - Or the Target is completely random and shares no informational connection with the Source.

3. Output Requirements:
* I will provide a JSON list of DNA-Protein pairs.
* You must return a RAW JSON object containing a list of results.
* The format must be strictly: `[{"id": 1, "prediction": "Coding"}, {"id": 2, "prediction": "Non-Coding"}, ...]`
* Do NOT provide explanations. Just the JSON array.
"""

# User Prompt: 核心修改点 —— 删除了 "triplet decoding"
# 替换为更抽象的 "information consistency" (信息一致性) 和 "pattern alignment" (模式对齐)
user_prompt = f"""Here is the JSON list of pairs to analyze.
Sentence 1 is the **DNA Source**.
Sentence 2 is the **Protein Target**.

**Do NOT rely on explicit biological rules (like codon tables).**
Instead, use your intuition about **"information consistency"** and **"latent pattern alignment"** to determine if the Protein is the correct representation of the DNA.

Data:
{json.dumps(prompt_data_list, indent=2)}
"""

In [7]:
# ==========================================
# 4. 调用 Volcengine API
# ==========================================
print("-" * 30)
print(f"Calling Volcengine Model: {MODEL_ID}...")

try:
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.1, # 低温以保证输出格式稳定
        top_p=0.9,
    )
    
    full_content = response.choices[0].message.content.strip()
    print("Response received.")
    # 打印前200个字符用于调试
    print(f"Response snippet: {full_content[:200]}...")

except Exception as e:
    print(f"API Call Failed: {e}")
    full_content = ""

------------------------------
Calling Volcengine Model: qwen3-max...
Response received.
Response snippet: [{"id": 1, "prediction": "Coding"}, {"id": 2, "prediction": "Coding"}, {"id": 3, "prediction": "Coding"}, {"id": 4, "prediction": "Non-Coding"}, {"id": 5, "prediction": "Non-Coding"}, {"id": 6, "predi...


In [8]:
# ==========================================
# 5. 解析结果与评估 (Coding 关系版本)
# ==========================================
import re
import json
from sklearn.metrics import accuracy_score, classification_report

def parse_llm_json(text):
    """提取并解析 JSON"""
    try:
        # 1. 尝试找被 ```json ... ``` 包裹的内容 (更稳健)
        code_block = re.search(r"```json\s*(\[.*?\])\s*```", text, re.DOTALL)
        if code_block:
            return json.loads(code_block.group(1))
        
        # 2. 如果没代码块，直接找最外层的 [...]
        match = re.search(r"\[.*\]", text, re.DOTALL)
        if match:
            return json.loads(match.group(0))
            
        # 3. 尝试直接解析
        return json.loads(text)
    except Exception as e:
        print(f"JSON Parsing Error: {e}")
        print(f"Raw Text Snippet: {text[:200]}...") # 打印前200字看看是啥
        return []

# 解析
predictions_list = parse_llm_json(full_content)

# ================= 关键修改区域 =================
# 1. 修改映射表：必须与 Prompt 里的输出要求一致
label_map = {
    "Coding": 1,      # 对应 Coding Relationship
    "Non-Coding": 0   # 对应 Non-Coding/Random
}
# ==============================================

y_true = []
y_pred = []

print("-" * 30)
if not predictions_list:
    print("Failed to parse JSON from model response.")
else:
    print(f"Parsed {len(predictions_list)} predictions.")
    
    for item in predictions_list:
        p_id = item.get('id')
        p_str = item.get('prediction')
        
        # 确保 ID 在真值表中，且模型预测的字符串在我们的映射表中
        if p_id in id_to_ground_truth:
            if p_str in label_map:
                y_true.append(id_to_ground_truth[p_id])
                y_pred.append(label_map[p_str])
            else:
                # 可选：打印一下模型造了什么新词，方便调试
                print(f"Warning: Unknown label '{p_str}' for ID {p_id}")

    # 输出最终指标
    if y_true:
        acc = accuracy_score(y_true, y_pred)
        print(f"\nFinal Accuracy: {acc:.2%}")
        print("\nClassification Report:")
        
        # 2. 修改报告显示的名称
        print(classification_report(
            y_true, 
            y_pred, 
            target_names=["Non-Coding (0)", "Coding (1)"]
        ))
        
        result_log = {
            "model": MODEL_ID,
            "accuracy": acc,
            "predictions": predictions_list
        }
    else:
        print("No valid matching IDs found. Check if 'id_to_ground_truth' keys match the JSON IDs.")

------------------------------
Parsed 100 predictions.

Final Accuracy: 83.00%

Classification Report:
                precision    recall  f1-score   support

Non-Coding (0)       1.00      0.66      0.80        50
    Coding (1)       0.75      1.00      0.85        50

      accuracy                           0.83       100
     macro avg       0.87      0.83      0.82       100
  weighted avg       0.87      0.83      0.82       100

